In [142]:
reload_lamb()

# Ad-hoc polymorphism via disjunctive types

The lambda notebook supports ad-hoc polymorphism via what are called disjunctive types.  A disjunctive type is effectively a set of types that unifies with another type if either (i) that type is disjunctive and there is overlap, or (ii) that type is a member of the disjunctive type.  If the result of unification would be a singleton, the resulting type is the unique member of that singleton.  Disjunctive types are written with square brackets and `|` as the separator.  They can be written recursively, but will be interpreted with that recursion flattened out.

**examples**
1. `[e|t]` and `[e|t]` unify to `[e|t]` (reflexivity)
2. Disjunctive types are normalized alphabetically, so for example `[t|e]` is normalized as `[e|t]`.  This ensures symmetry.
3. `[e|t|<e,t>]` and `[e|t|n]` unify to `[e|t]`
4. `[e|t]` and `e` unify to `e`.
5. `[e|t]` and `[t|n]` unify to `t`.
6. `[e]` is invalid.  So is `[e|e]`, etc.
7. `[e|e|t]` is interpreted as just `[e|t]`.
8. `[e|[n|t]]` is interpreted as just `[e|n|t]`

One further constraint is that type variables (or types using them) cannot be disjoined.

In [143]:
unify(tp("[e|t]"), tp("[e|t]"))

[e|t]

In [144]:
tp("[t|e]")

[e|t]

In [145]:
unify(tp("[e|t|<e,t>]"), tp("[e|t|n]"))

[e|t]

In [146]:
unify(tp("[e|t]"), tp("e"))

e

In [147]:
unify(tp("[e|t]"), tp("[t|n]"))

t

In [148]:
try:
    tp("[e]")
except types.TypeParseError as e:
    print(e)

Disjunctive type must have multiple unique disjuncts at point '!here![e]'


In [149]:
try:
    tp("[e|e]")
except types.TypeParseError as e:
    print(e)

Disjunctive type must have multiple unique disjuncts at point '!here![e|e]'


In [150]:
tp("[e|e|t]")

[e|t]

In [151]:
tp("[e|[n|t]]")

[e|n|t]

## Using disjunctive types in expressions

Disjunctive types can be straightforwardly used as term types, and all appropriate type inference should happen as expected.  This is the most typical use in linguistics, where someone might want to express that a (not-further-defined) constant is polymorphic in some way.

In [152]:
%%lamb 
||equals|| = L x_[e|n] : Equivalent(x)
x = x_e

INFO (meta): Coerced guessed type for 'Equivalent_t' into <[e|n],t>, to match argument 'x_[e|n]'


$[\![\mathbf{\text{equals}}]\!]^{}_{\left\langle{}{\left[e\mid{}n\right]},t\right\rangle{}} \:=\: $$\lambda{} x_{{\left[e\mid{}n\right]}} \: . \: {Equivalent}_{\left\langle{}{\left[e\mid{}n\right]},t\right\rangle{}}({x})$<br />
${x}_{e}\:=\:{x}_{e}$

In [153]:
equals.content(x) # forces narrowing of the argument type

((λ x_e: Equivalent_<e,t>(x_e)))(x_e)

In [154]:
%te P_<[e|n],t>(x_[n|t]) # forces narrowing of both types

P_<n,t>(x_n)

In [155]:
%te (L x_[e|n] : P_<[e|n],t>(x))(C_n) # forces narrowing including of both x and, indirectly, the predicate's type

((λ x_n: P_<n,t>(x_n)))(C_n)

While you can't use variable types in disjunctive types, they will unify with variable types correctly:

In [156]:
te("(L x_[e|n] : P_<[e|n|t],t>(x))(C_X)")

((λ x_[e|n]: P_<[e|n],t>(x_[e|n])))(C_[e|n])

The other way to use disjunctive types is via a `Disjunctive` expression.  This gives, basically, a disjunctive formula, where type inference will choose between expressions depending on type.  You can think of this (in a roundabout way) as something like a `dict`/hashtable mapping types to expressions.  This may help understand some odd corner cases -- `Disjunctive` expressions have to ensure that every non-disjunctive type is mapped to a unique expression.  A disjunctive expression displays as the function Disjunctive, with the type annotated as a superscript.

In [157]:
x = %te Disjunctive(A_e, B_n, C_t)
x

Disjunctive(A_e,B_n,C_t)

Adjusting the type, either explicitly or implicitly, can narrow down the formula, resulting in either another Disjunctive type, or just some TypedExpr.  (Or `None`, if the adjustment fails.)

In [158]:
x.try_adjust_type(tp("e"))

A_e

In [159]:
x.try_adjust_type(tp("[e|t]"))

Disjunctive(A_e,C_t)

In [160]:
x.try_adjust_type(tp("[e|t]")).try_adjust_type(tp("n"))

In [161]:
te("Disjunctive(x_e, y_[<e,t>|n], z_t)").try_adjust_type(tp("[e|t]"))

Disjunctive(x_e,z_t)

Disjuncts most have unique types, or more generally, must lead deterministically to unique formulas for every type refinement.  So the following cases are out.  The second, more complicated case, would lack a unique refinement at type `e`.

In [162]:
%te Disjunctive(x_e, y_e)

ERROR (parsing): Parsing of typed expression failed with exception:
ERROR (parsing): Disjunctive type must have multiple unique disjuncts


In [163]:
%te Disjunctive(x_e, y_e, z_[e|t])

ERROR (parsing): Parsing of typed expression failed with exception:
ERROR (parsing): Disjoined expressions must determine unique types (type e appears duplicated in expression 'y_e')


While disjunctive types cannot recurse, Disjunction expressions can recurse or involve multiple disjunctive types, as long as they obey the other type constraints.  (There still must be a unique refinement.)  Type adjustment will result in flattening of Disjunctions.

In [164]:
r = %te Disjunctive(x_e, Disjunctive(y_n, z_t))
r

Disjunctive(x_e,Disjunctive(y_n,z_t))

In [165]:
r.try_adjust_type(tp("n"))

y_n

In [166]:
r.try_adjust_type(tp("[e|n]"))

Disjunctive(x_e,y_n)

In [167]:
r.try_adjust_type(tp("[n|t]"))

Disjunctive(y_n,z_t)

In [168]:
%te Disjunctive(x_e, y_[<e,t>|n])

Disjunctive(x_e,y_[<e,t>|n])

Quite complex things are possible with disjunctive types, once you combine them with other tools.  Keep in mind, though, that type inference on variables must be stable across all instances of that variable's use.

In [195]:
f = %te L x_e : Disjunctive(x_[e|n], False_t)
f

(λ x_e: Disjunctive(x_e,False_t))

In [196]:
%te reduce (L x_e : Disjunctive(x_[e|n], y_t))(A_e)

Disjunctive(A_e,y_t)

In [197]:
%te reduce (L x_[e|t] : P(x) & x)((L x_e : Disjunctive(x_[e|n], y_t))(A_e))

INFO (meta): Coerced guessed type for 'P_t' into <[e|t],t>, to match argument 'x_[e|t]'


(P_<t,t>(y_t) & y_t)